In [2]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
from glob import glob

import matplotlib.pyplot as plt

import rioxarray as rio
from shapely.geometry import mapping

%matplotlib inline
# %matplotlib widget

from shapely.geometry import mapping

In [3]:
# wrap locating of the median points into a function: 

### (1) read .shp

In [14]:
hma = gpd.read_file('HMA_regions/HMA_regions.shp')
# hma.plot()

# as dataframe
hma_df = pd.DataFrame(hma) 

### (2) read climate

In [5]:
climate = xr.open_mfdataset('*.nc', decode_coords="all")
# cnange units (and overwrite the metadata with the units after)

# ------ UNITS ------

#convert temperature K to C
climate['t2m']=climate.t2m-273.15
# precipotation m to mm 
climate['tp']=climate.tp * 1000
# radiation j/m2 to w/m2
# SSR [W/m2] = SSR [J/m^2] / (3600 seconds)
climate['ssrd'] = climate.ssrd / 3600

# ------ CRS ----------

# add crs to the dataset (netcdf file) - maybe this is not even needed 
climate = climate.rio.write_crs('EPSG:4326')



In [6]:
#

climate_median = climate.median(dim='time', keep_attrs = True, skipna=True)

# climate_median

In [15]:
# hma = hma[hma['Name'] == hma_df.Name[21]]

hma = hma[:3]

In [16]:
hma

,Name,descriptio,timestamp,begin,end,altitudeMo,tessellate,extrude,visibility,drawOrder,icon,snippet,geometry
0,Eastern Hindu Kush,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",None,None,None,clampToGround,-1,0,-1,None,None,None,"POLYGON Z ((73.45169 36.98397 0.00000, 73.4459..."
1,Western Himalaya,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",None,None,None,clampToGround,-1,0,-1,None,None,None,"POLYGON Z ((75.78745 35.31560 0.00000, 75.7874..."
2,Eastern Himalaya,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",None,None,None,clampToGround,-1,0,-1,None,None,None,"POLYGON Z ((95.22303 29.82187 0.00000, 95.2135..."


### (3) function: 

In [17]:
# function for median: 

def median_ceil(values):
    values = sorted(values)
    values_len = len(values)
    middle = values_len//2
    return values[middle]


In [10]:
# hma_df.head()

In [20]:
# polygons = hma.geometry.apply(mapping)
# hma_df.values

#  geometries = geodf.geometry.apply(mapping)
# hma.geometry
# polygons

0    POLYGON Z ((73.45169 36.98397 0.00000, 73.4459...
1    POLYGON Z ((75.78745 35.31560 0.00000, 75.7874...
2    POLYGON Z ((95.22303 29.82187 0.00000, 95.2135...
Name: geometry, dtype: geometry

In [27]:
type(hma)

geopandas.geodataframe.GeoDataFrame

In [19]:
%%time
# for every geometry in hma

name = []
value = []
lats = []
lons = []


# for r in polygons:
for geometry in hma.geometry:
    
    # clip:
    clipped = climate_median.rio.clip(hma.geometry, hma.crs, drop=True, invert=False)
    # clipped = climate_median.rio.clip(geometry, drop=True, invert=False)

    
    # sanity check
    # clipped.t2m.plot()
    
    # to dataframe:
    df = clipped.to_dataframe().reset_index()
    df = df.dropna().drop_duplicates()
    
    # median vallues for a parameter: 
    median_value = median_ceil(values=df.t2m)
    # print('temp median value', median_value)
    
    # find index of this value:
    row = df[df.t2m == median_value]
    lat = row.latitude.values[0]
    lon = row.longitude.values[0]
    
    print(lat)
    
    #add:
    value = np.append(value, median_value)
    lats = np.append(lats, lat)
    lons = np.append(lons, lon)

    # print('name:', name)
    # print('lat', lat)
    # print('lon', lon)

    

27.75


KeyboardInterrupt: 

lats
lons
value
name

In [ ]:
table = pd.DataFrame([name, lats, lons, value]).transpose()

In [ ]:
table

In [ ]:
name

In [ ]:
lons